In [1]:
#Thèmes possibles: chesterish, grade3, gruvboxd, gruvboxl, monokai, oceans16, onedork, solarizedd, solarizedl
#Thèmes préférés: clair: grade3, foncés: chesterish
#-T: toolbar
# !jt -r :reset de tout

!jt -t chesterish -T -N -kl -fs 15 -nfs 15 -tfs 15 -dfs 15 -cellw 80% 
!jt -r

Reset css and font defaults in:
/home/waup/.jupyter/custom &
/home/waup/.local/share/jupyter/nbextensions


In [2]:
def print_dimensions(file):
  print('Global dimensions:')
  for dim in file.dimensions.values():
    if (dim.isunlimited()):
      print("  {0: <8} = {1} (unlimited)".format(dim.name,len(dim)))
    else:
      print("  {0: <8} = {1}".format(dim.name,len(dim)))

In [3]:
def print_field(file,fld):
    print('Variable : ',fld)
    print('Type : ',file.variables[fld].dtype)
    print('Liste des dimensions : ')
    for dim in file.variables[fld].dimensions:
        print('  ',dim)
    print('Taille(s) : ')
    for sz in file.variables[fld].shape:
          print('  ',sz)
    if ( hasattr(file.variables[fld], 'coordinates') ):
        print('Coordonnées auxiliaires :')
        print('  ',file.variables[fld].coordinates)
    print('Attributs :')
    for md in file.variables[fld].ncattrs():
      print("  {0: <20} = {1}".format(md,getattr(file.variables[fld],md)))


In [4]:
def print_group(file,grpname):
    grp=file[grpname]
    print('Chemin groupe HDF5 : ',grpname)
    print('Groupe HDF5 : ',grp.name)
    print('Attributs :')
    for att in grp.ncattrs():
      print("  {0: <30} = {1}".format(att, getattr(grp, att)))
    if ( hasattr(grp, 'dimensions') ):
      print('Liste des dimensions : ')
      for dim in grp.dimensions:
        print('  ',dim)
    print('Sous-groupes:')
    for sgrp in grp.groups:
      print('  ',sgrp)
    print('Variables:')
    for var in grp.variables:
      print('  ',var)
    print('----------------------------------------------------------------------------------')

# Entrées-sorties dans Méso-NH <a class="tocSkip">

## Introduction et plan

Cette présentation a pour but d'expliquer la gestion des entrées-sorties dans Méso-NH. Elle est basée principalement sur la version 5.5.0.

La présentation est découpée en chapitre selon le plan suivant :
*  [Historique](#Historique)
*  [Types de fichiers](#types_fichiers)
*  [Structure fichiers netCDF](#structure_fichiers_netcdf)
*  [Namelistes](#Namelistes)
*  [Code source](#code_source)

## Historique / évolutions majeures

### Avant-hier : Méso-NH 5.4.x

Méso-NH 5.4 : apports principaux :
*  Distinction entre *backups* (reprises) et *outputs* (sorties "fréquentes" avec choix des champs)
*  Respect convention CF (netCDF uniquement, encore perfectible)
*  Ajout de métadonnées pour les différents champs (netCDF)

### Hier : Méso-NH 5.5.x

L'organisation des fichiers diachroniques ("000") a été entièrement revue.
Les sorties pour les bilans, bilans LES, avions, ballons, profileurs, stations et séries temporelles ont été modifiées.

*  La structure est basée sur les groupes des fichiers HDF5. Ils peuvent être vus comme des répertoires dans les systèmes de fichiers.
*  Namelistes NAM\_BU\_R* simplifiées (seulement 2 entrées)
* Ajout de nouvelles dimensions

### Hier : Méso-NH 5.6.x

Apports marquants pour les versions 5.6 :

* Réorganisation de la gestion des variables scalaires (variables utilisateurs, microphysiques, électricité, chimie, traceurs Lagrangiens, polluants passifs, sels, poussières...)
    * correction de bugs, d'incohérences et d'oublis (variables manquantes) dans les fichiers
    * changements dans le nommage
    * homogénéisation des unités physiques
    * métadonnées plus explicites
* Capteurs (avions, ballons, stations et profileurs) : réorganisation complète
    * utilisation de namelistes et de fichiers CSV pour spécifier toutes leurs caractéristiques
    * correction d'oublis et champs supplémentaires dans les sorties
    * optimisation des communications MPI (parallélisme) et de la consommation mémoire
* Pour les développeurs : mise en place d'un constructeur personnalisé (*custom constructor*, fonctionnalité Fortran 2003) pour les métadonnées des champs

### Aujourd'hui : Méso-NH 5.7.x

Apports marquants pour les versions 5.7 : peu de changements importants par rapport aux version 5.6

* Outputs (sorties "fréquentes") : compression avec pertes

### Demain

Evolutions futures :
*  I/O parallèles revisitées
*  Réorganisation interne et amélioration du code
*  Poursuite des améliorations sur la structure des fichiers


<a id='types_fichiers'></a>
## Types de fichiers

### Fichiers d'entrées

*  Namelistes (paramètres physiques des modèles et paramètres d'exécution)
*  Fichiers de démarrage initiaux (générés par PREP_IDEAL_CASE, PREP_REAL_CASE...)
*  *Backups* (si reprise)
*  Divers (liste des stations, caractéristiques éoliennes...)

Remarque : liste non exhaustive, ces fichiers n'étant pas tous le sujet de cette formation

### Fichiers de sortie

*  *Backups* (fichiers de reprise) synchrones : tous les champs nécessaires pour un redémarrage + autres à un instant donné de la simulation
*  Diachroniques ("000") : contiennent tous les bilans + autres données
*  Outputs (sorties "fréquentes") : sélection de champs
*  *OUTPUT_LISTINGn* : messages
*  Fichiers .des (aussi dans les entrées, associés à autres fichiers)
*  Divers (éclairs...)

Remarque : liste non exhaustive, ces fichiers n'étant pas tous le sujet de cette formation

<a id='structure_fichiers_netcdf'></a>
## Structure fichiers netCDF

Méso-NH peut travailler avec des fichiers aux formats LFI ou netCDF.

Sur ce support, seul le contenu des fichiers netCDF sera passé en revue. Le format LFI est le format utilisé historiquement par Méso-NH et est nettement moins riche que netCDF.

*  Les fichiers netCDF et LFI (sauf pour les sorties "fréquentes") sont accompagnés d'un fichier texte avec comme extension *.des*. Ces derniers contiennent le contenu des namelistes utilisées.
*  Les fichiers netCDF essayent de respecter les [conventions CF](http://cfconventions.org/) pour un meilleur support des outils standards de traitement de données et de visualisation, ainsi que pour l'interopérabilité.

### Nommage des fichiers

#### Fichiers de *backup*

Nom : ***CEXP.n.CSEG.bbb.ext***

avec

*  *CEXP* le nom de l'expérimentation (5 caractères),
*  *n* le numéro de modèle,
*  *CSEG* le nom du segment (5 caractères),
*  *bbb* le numéro du *backup* (3 chiffres, commence à 001 au début du segment),
*  *ext* l'extension du fichier (nc, lfi ou des)

Cas particulier si découpage en niveaux verticaux (entrées-sorties parallèles) :

Nom : ***CEXP.n.CSEG.bbb.Zlll.ext***

avec *Zlll* pour les numéros du fichier contenant les champs 3D découpés par niveaux (de 001 à nombre de processus utilisés pour le découpage)

#### Fichiers d'*output*

Nom : ***CEXP.n.CSEG.OUT.ooo.ext***

avec

*  *CEXP* le nom de l'expérimentation (5 caractères),
*  *n* le numéro de modèle,
*  *CSEG* le nom du segment (5 caractères),
*  *ooo* le numéro du *sortie* (3 chiffres, commence à 001 au début du segment),
*  *ext* l'extension du fichier (nc, lfi ou des)

Cas particulier si découpage en niveaux verticaux (entrées-sorties parallèles) :

Nom : **CEXP.n.CSEG.OUT.ooo.Zlll.ext**

avec *Zlll* pour les numéros du fichier contenant les champs 3D découpés par niveaux (de 001 à nombre de processus utilisés pour le découpage)

#### Fichiers diachroniques

Nom : ***CEXP.n.CSEG.000.ext***

avec

*  *CEXP* le nom de l'expérimentation (5 caractères),
*  *n* le numéro de modèle,
*  *CSEG* le nom du segment,
*  *ext* l'extension du fichier (nc, lfi ou des)

Cas particulier si découpage en niveaux verticaux (entrées-sorties parallèles) :

Nom : ***CEXP.n.CSEG.000.Zlll.ext***

avec *Zlll* pour les numéros du fichier contenant les champs 3D découpés par niveaux (de 001 à nombre de processus utilisés pour le découpage)

### Fichiers de backups et d'outputs

#### Ouverture fichier

In [5]:
import netCDF4 as nc

fl_backup = nc.Dataset( 'data/REUNI.1.00A20.004.nc', 'r' )

#### Attributs

In [6]:
print('Global attributes:')
for att in fl_backup.ncattrs():
  print("  {0: <30} = {1}".format(att, getattr(fl_backup, att)))

Global attributes:
  Conventions                    = CF-1.10 COMODO-1.4
  MNH_REAL                       = 8
  MNH_INT                        = 4
  MNH_REDUCE_DIMENSIONS_IN_FILES = 1
  history                        = 2023-10-05T10:58:22+0200: /home/waup/MNH/MNH-56-branch/src/dir_obj-LXgfortran-R8I4-MNH-V5-6-1-MPIAUTO-DEBUG/MASTER/MESONH
  MNH_cleanly_closed             = yes


#### Dimensions globales

In [7]:
print_dimensions(fl_backup)

Global dimensions:
  ni       = 74
  nj       = 82
  ni_u     = 74
  nj_u     = 82
  ni_v     = 74
  nj_v     = 82
  level    = 52
  level_w  = 52
  time     = 1 (unlimited)
  SW_bands = 6
  LW_bands = 16
  size3    = 3
  char16   = 16
  char32   = 32
  size12   = 12
  size2    = 2


Remarques :
*  lorsqu'une dimension n'est pas trouvée lors de l'exécution, elle est créée automatiquement par Méso-NH avec un nom préfixé par *char* suivi par sa taille (par multiple de 16) pour les chaînes de caractères et *size* suivi de sa taille pour les autres types
*  d'autres dimensions globales existent dans les fichiers diachroniques (voir plus loin)

#### Coordonnées 1D

Elles ont comme nom, le même que celui d'une dimension.

Attributs utiles :
*  *axis* : *X* ou *Y* pour les coordonnées horizontales, *Z* pour les verticales et *T* pour le temps
*  *c_grid_axis_shift* (venant de la norme COMODO) : décalage par rapport à autres coordonnées sur le même axe
* *c_grid_dynamic_range* : donne l'intervalle de numéros d'indices appartenant au domaine physique (et permet donc de retirer les mailles fantômes)


In [8]:
print_field(fl_backup,'ni_u')

Variable :  ni_u
Type :  float64
Liste des dimensions : 
   ni_u
Taille(s) : 
   74
Attributs :
  long_name            = x-dimension of the grid at u location
  standard_name        = projection_x_coordinate_at_u_location
  units                = m
  axis                 = X
  c_grid_axis_shift    = -0.5
  c_grid_dynamic_range = 2:74


In [9]:
print_field(fl_backup,'level')

Variable :  level
Type :  float64
Liste des dimensions : 
   level
Taille(s) : 
   52
Attributs :
  long_name            = position z in the transformed space
  units                = m
  axis                 = Z
  positive             = up
  c_grid_axis_shift    = 0.0
  c_grid_dynamic_range = 2:51
  formula_terms        = s: level height: ZTOP orog: ZS
  formula_definition   = z(n,k,j,i)=s(k)*(height-orog(j,i))/height+orog(j,i)
  computed_standard_name = altitude


In [10]:
print_field(fl_backup,'time')

Variable :  time
Type :  float64
Liste des dimensions : 
   time
Taille(s) : 
   1
Attributs :
  long_name            = time axis
  standard_name        = time
  units                = seconds since 2000-01-01 00:00:00 +0:00
  axis                 = T
  calendar             = standard


#### Variables auxiliaires de coordonnées

Variables de coordonnées 2D basées sur d'autres coordonnées

In [11]:
print_field(fl_backup,'latitude_u')

Variable :  latitude_u
Type :  float64
Liste des dimensions : 
   nj_u
   ni_u
Taille(s) : 
   82
   74
Attributs :
  standard_name        = latitude_at_u_location
  long_name            = latitude at u location
  units                = degrees_north
  grid                 = 2
  comment              = X_Y_latitude at u point
  _FillValue           = 9.969209968386869e+36
  valid_min            = -1e+36
  valid_max            = 1e+36


#### Champs

In [12]:
print_field(fl_backup,'BUGFIX')

Variable :  BUGFIX
Type :  int32
Liste des dimensions : 
Taille(s) : 
Attributs :
  long_name            = MesoNH bugfix number


In [13]:
print_field(fl_backup,'UT')

Variable :  UT
Type :  float64
Liste des dimensions : 
   time
   level
   nj_u
   ni_u
Taille(s) : 
   1
   52
   82
   74
Coordonnées auxiliaires :
   latitude_u longitude_u
Attributs :
  standard_name        = x_wind
  long_name            = UT
  units                = m s-1
  grid                 = 2
  comment              = X_Y_Z_U component of wind
  coordinates          = latitude_u longitude_u
  _FillValue           = 9.969209968386869e+36
  valid_min            = -1e+36
  valid_max            = 1e+36


#### Fichiers découpés en niveaux verticaux (I/O parallèles)

Si les entrées-sorties parallèles sont activées, les champs 3D sont découpés dans des sous-fichiers (avec des *Znnn* dans leur nom).

Dans l'exemple ci-dessous, on travaille avec un fichier d'*output* (sorties "fréquentes") et ses sous-fichiers avec compression et réduction de la précision des nombres en virgule flotantte.

In [14]:
fl_output_master = nc.Dataset( 'data/REUNI.1.00A20.OUT.009.nc', 'r' )
fl_output_slave = nc.Dataset( 'data/REUNI.1.00A20.OUT.009.Z001.nc', 'r' )

In [15]:
print_field(fl_output_master,'UT')

Variable :  UT
Type :  float32
Liste des dimensions : 
Taille(s) : 
Coordonnées auxiliaires :
   latitude_u longitude_u
Attributs :
  standard_name        = x_wind
  long_name            = UT
  units                = m s-1
  grid                 = 2
  comment              = X_Y_Z_U component of wind
  coordinates          = latitude_u longitude_u
  _FillValue           = 9.969209968386869e+36
  valid_min            = -9.999999616903162e+35
  valid_max            = 9.999999616903162e+35
  split_variable       = yes
  split_mode           = Z
  split_nblocks        = 52
  split_nfiles         = 8
  split_distribution   = round-robin
  ndims                = 3
  dims                 = [74 82 52]
  time_dependent       = yes


In [16]:
print_field(fl_output_slave,'UT0009')

Variable :  UT0009
Type :  float32
Liste des dimensions : 
   level
   nj_u
   ni_u
Taille(s) : 
   52
   82
   74
Coordonnées auxiliaires :
   latitude_u longitude_u
Attributs :
  standard_name        = x_wind_at_level_0009
  long_name            = UT at level 0009
  units                = m s-1
  grid                 = 2
  comment              = X_Y_Z_U component of wind
  coordinates          = latitude_u longitude_u
  _FillValue           = 9.969209968386869e+36
  valid_min            = -9.999999616903162e+35
  valid_max            = 9.999999616903162e+35


### Fichiers diachroniques (*000*)

Ils contiennent les sorties pour :

*  bilans (*budgets*)
*  bilans LES
*  avions
*  ballons
*  profileurs
*  stations
*  séries temporelles

La structure est basée sur les groupes des fichiers HDF5. Ils peuvent être vu comme des répertoires dans les systèmes de fichiers.
Il y a jusqu'à 7 niveaux (*levels*) hiérarchiques. Certains peuvent être regroupés pour des questions de lisibilité. La liste des niveaux dans le groupe est donnée via l'attribut *levels*.
  
Niveaux dans les fichiers diachroniques :

*  *category* : valeurs possibles : 'Budgets', 'LES\_budgets', 'Flyers' (pour les avions et les ballons), 'Profilers', 'Stations' and 'Time\_series'
*  *subcategory* : utilisé seulement pour les 'Flyers'
*  *group*
*  *shape*
*  *timeavg* : données moyennées ou pas dans le temps
*  *norm* : données normalisées ou pas
*  *mask*

Les valeurs des champs sont toujours stockées dans le dernier niveau. Il est donc possible d'avoir plusieurs champs avec le même nom mais à des emplacements différents. Leur distinction se fait selon les caractéristiques des différents niveaux.

La plupart des dimensions sont des dimensions globales car elles peuvent potentiellement servir à plusieurs endroits. Néanmoins, certaines peuvent être locales à un groupe HDF5. C'est le cas de la dimension temporelle pour les *flyers* qui est spécifique à chacun d'entre eux.

#### Ouverture fichier

In [17]:
fl_dia = nc.Dataset( 'data/REUNI.1.00A20.000.nc', 'r' )

#### Attributs

In [18]:
print('Global attributes:')
for att in fl_dia.ncattrs():
  print("  {0: <30} = {1}".format(att, getattr(fl_dia, att)))

Global attributes:
  Conventions                    = CF-1.10 COMODO-1.4
  MNH_REAL                       = 8
  MNH_INT                        = 4
  MNH_REDUCE_DIMENSIONS_IN_FILES = 1
  history                        = 2023-10-05T10:54:03+0200: /home/waup/MNH/MNH-56-branch/src/dir_obj-LXgfortran-R8I4-MNH-V5-6-1-MPIAUTO-DEBUG/MASTER/MESONH
  MNH_cleanly_closed             = yes


#### Dimensions globales

In [19]:
print_dimensions(fl_dia)

Global dimensions:
  ni       = 74
  nj       = 82
  ni_u     = 74
  nj_u     = 82
  ni_v     = 74
  nj_v     = 82
  level    = 52
  level_w  = 52
  one      = 1
  real_imaginary = 2
  pair     = 2
  cart_ni  = 50
  cart_nj  = 54
  cart_ni_u = 50
  cart_nj_u = 54
  cart_ni_v = 50
  cart_nj_v = 54
  time_budget = 20
  time_les = 20
  time_les_avg = 2
  level_les = 4
  nspectra_2pts_ni = 71
  nspectra_2pts_nj = 79
  nspectra_spec_ni = 70
  nspectra_spec_nj = 78
  nspectra_level = 2
  time_profiler = 21
  time_station = 41
  series_level = 50
  series_level_w = 50
  time_series = 12
  size3    = 3
  char16   = 16
  char32   = 32


Certains champs peuvent être moyennés sur le temps. Dans ce cas, une variable supplémentaire est ajoutée pour spécifier les plages de temps.

Exemple :

In [20]:
print_field(fl_dia,'time_les_avg')
print('Valeurs : ',fl_dia.variables['time_les_avg'][:])

Variable :  time_les_avg
Type :  float64
Liste des dimensions : 
   time_les_avg
Taille(s) : 
   2
Attributs :
  long_name            = time axis for LES budget time averages
  standard_name        = time
  units                = seconds since 2000-01-01 00:00:00 +0:00
  axis                 = T
  calendar             = standard
  bounds               = time_les_avg_bounds
Valeurs :  [120. 300.]


In [21]:
print_field(fl_dia,'time_les_avg_bounds')

print('Valeurs : ',fl_dia.variables['time_les_avg_bounds'][:])

Variable :  time_les_avg_bounds
Type :  float64
Liste des dimensions : 
   time_les_avg
   pair
Taille(s) : 
   2
   2
Attributs :
Valeurs :  [[ 40. 200.]
 [200. 400.]]


#### Structure des niveaux

In [22]:
print('Liste des groupes HDF5 dans la racine du fichier :')
for gr in fl_dia.groups:
    print(gr)

Liste des groupes HDF5 dans la racine du fichier :
Budgets
Time_series
Flyers
Stations
Profilers
LES_budgets


In [23]:
print('Liste de tous les groupes HDF5 dans le fichier :')
for lv1 in fl_dia.groups.values():
    print(lv1.path)
    for lv2 in lv1.groups.values():
        print(lv2.path)
        for lv3 in lv2.groups.values():
            print(lv3.path)
            for lv4 in lv3.groups.values():
                print(lv4.path)
                for lv5 in lv4.groups.values():
                    print(lv5.path)
                    for lv6 in lv5.groups.values():
                        print(lv6.path)
                        for lv7 in lv6.groups.values():
                            print(lv7.path)

Liste de tous les groupes HDF5 dans le fichier :
/Budgets
/Budgets/RhodJ
/Budgets/UU
/Budgets/VV
/Budgets/TK
/Time_series
/Time_series/TSERIES
/Time_series/TSERIES/GLOB
/Time_series/TSERIES/LAND
/Time_series/TSERIES/SEA
/Time_series/ZTSERIES
/Time_series/ZTSERIES/GLOB
/Time_series/ZTSERIES/LAND
/Time_series/ZTSERIES/SEA
/Time_series/XTSERIES01
/Flyers
/Flyers/Constant_volume_balloons
/Flyers/Constant_volume_balloons/CVB1MOBI
/Flyers/Constant_volume_balloons/CVB1MOBI/Point
/Flyers/Constant_volume_balloons/CVB1MOBI/Vertical_profile
/Flyers/Constant_volume_balloons/CVB2MOBI
/Flyers/Constant_volume_balloons/CVB2MOBI/Point
/Flyers/Constant_volume_balloons/CVB2MOBI/Vertical_profile
/Flyers/Aircrafts
/Flyers/Aircrafts/DIMO19A
/Flyers/Aircrafts/DIMO19A/Point
/Flyers/Aircrafts/DIMO19A/Vertical_profile
/Stations
/Stations/STATPW
/Stations/SNUM2
/Profilers
/Profilers/CAAM
/Profilers/CAAM/Vertical_profile
/Profilers/CAAM/Point
/Profilers/CRAU
/Profilers/CRAU/Vertical_profile
/Profilers/CRAU/Point


#### Budgets

##### Liste des dimensions et coordonnées si utilisation d'une boîte cartésienne (CBUTYPE='CART')

*  *cart\_ni, cart\_ni\_u, cart\_ni\_v*: points of the budget cartesian box in the I-direction (for mass-, u- and v-points respectively)
*  *cart\_nj, cart\_nj\_u, cart\_nj\_v*: points of the budget cartesian box in the J-direction (for mass-, u- and v-points respectively)
*  *cart\_level, cart\_level\_w*: vertical levels of the budget cartesian box (for mass- and w-points respectively)
*  *time\_budget*: middle of each time interval used to compute temporally averaged budgets
*  *time\_budget\_bounds* (no dimension, only for coordinates): boundaries of each time interval

##### Liste des dimensions et coordonnées si utilisation de masques horizontaux (CBUTYPE='MASK')

*  *mask\_level, mask\_level\_w*: vertical levels of the horizontal budget masks (for mass- and w-points respectively)
*  *time\_budget*: middle of each time interval used to compute temporally averaged budgets
*  *time\_budget\_bounds* (no dimension, only for coordinates): boundaries of each time interval
*  *nbumask* (only dimension, no corresponding coordinate): number of horizontal masks

##### Description des niveaux (groupes HDF5)

*  *category*: 'Budgets'
*  *subcategory*: not used (merged into category level)
*  *group*: possible values:
      * 'UU': budget for U
      * 'VV': budget for V
      * 'WW': budget for W
      * 'TH': budget for potential temperature
      * 'TK': budget for turbulent kinetic energy
      * 'RV': budget for water vapor mixing ratio
      * 'RC': budget for cloud water mixing ratio
      * 'RR': budget for rain water mixing ratio
      * 'RI': budget for cloud ice mixing ratio
      * 'RS': budget for snow/aggregate mixing ratio
      * 'RG': budget for graupel mixing ratio
      * 'RH': budget for hail mixing ratio
      * CSVNAMES: budget for scalar variables (names are taken from the CSVNAMES array)
      * 'RhodJ': mass of dry air contained in the mesh cells
*  *shape*: 'Cartesian' or 'Mask' (not used, merged into group level)
*  *timeavg*: 'Time\_averaged' (not used, merged into group level)
*  *norm*: 'Not\_normalized' (not used, merged into group level)
*  *mask*: not used (value set to reference the name of the variable used to store the masks if CBUTYPE='MASK', merged into group level)

Remarques :

*  if CBUTYPE='MASK', the masks are stored in the variable named 'MASKS' (stored in the root level of the diachronic file)
*  some of the dimensions might not appear if there is averaging in the corresponding direction (see LBU\_ICP, LBU\_JCP and LBU\_KCP options of the NAM\_BUDGET namelist)
*  some metadata is also available in the file and provide information about the boundaries of the cartesian box or masks, averaging in the different dimensions...

In [24]:
print_group(fl_dia,'/Budgets')

Chemin groupe HDF5 :  /Budgets
Groupe HDF5 :  Budgets
Attributs :
  levels                         = category subcategory
  category                       = Budgets
  comment                        = Level for the different budgets
  subcategory                    = 
Liste des dimensions : 
Sous-groupes:
   RhodJ
   UU
   VV
   TK
Variables:
----------------------------------------------------------------------------------


In [25]:
grp=fl_dia['/Budgets']

grpname='Budgets'
file=fl_dia

for sgrpname in grp.groups:
    sgrpnamefull='/'+grpname+'/'+sgrpname
    print_group(file,sgrpnamefull)

Chemin groupe HDF5 :  /Budgets/RhodJ
Groupe HDF5 :  RhodJ
Attributs :
  levels                         = group shape timeavg norm mask
  group                          = RhodJ
  comment                        = mass of dry air contained in the mesh cells
  shape                          = Cartesian
  moving                         = no
  min_I_index_in_physical_domain = 15
  max_I_index_in_physical_domain = 64
  min_J_index_in_physical_domain = 15
  max_J_index_in_physical_domain = 68
  min_K_index_in_physical_domain = 2
  max_K_index_in_physical_domain = 50
  averaged_in_the_I_direction    = no
  averaged_in_the_J_direction    = no
  averaged_in_the_K_direction    = yes
  time_averaged                  = yes
  normalized                     = no
  mask                           = 
Liste des dimensions : 
Sous-groupes:
Variables:
   RhodJX
   RhodJY
   RhodJS
----------------------------------------------------------------------------------
Chemin groupe HDF5 :  /Budgets/UU
Groupe HDF5

In [26]:
print_field(fl_dia['/Budgets/UU'],'ADV')

Variable :  ADV
Type :  float64
Liste des dimensions : 
   time_budget
   cart_nj_u
   cart_ni_u
Taille(s) : 
   20
   54
   50
Attributs :
  long_name            = advection
  units                = m s-2
  grid                 = 2
  comment              = Budget of momentum along X axis: advection source term
  _FillValue           = 9.969209968386869e+36
  valid_min            = -1e+36
  valid_max            = 1e+36


#### Budgets LES

##### Liste des dimensions

*  *time\_les*: sampling times
*  *time\_les\_avg*: middle of each time interval used to compute temporally averaged LES budgets
*  *time\_les\_avg\_bounds* (no dimension, only for coordinates): boundaries of each time\_les\_avg interval
*  *level\_les*: vertical levels
*  *nsv* (only dimension, no corresponding coordinate): number of scalar variables
*  *npdf* (only dimension, no corresponding coordinate): number of PDF intervals (see NAM\_PDF namelist)
*  *nspectra\_2pts\_ni, nspectra\_2pts\_nj* (only dimension, no corresponding coordinate): number of horizontal wavelengths for non-local LES budgets (2 points correlations) in I and J dimensions
*  *nspectra\_spec\_ni, nspectra\_spec\_nj* (only dimension, no corresponding coordinate): number of horizontal wavelengths for LES spectra budgets in I and J dimensions
*  *nspectra\_level*: vertical levels for non-local LES budgets

##### Description des niveaux (groupes HDF5)

*  *category*: 'LES\_budgets'
*  *subcategory*: not used (merged into category level)
*  *group*: possible values:
      * 'Mean': mean vertical profiles of the model variables
      * 'PDF'
      * 'Resolved': mean vertical profiles of the resolved fluxes, variances and covariances
      * 'Subgrid': mean vertical profiles of the subgrid fluxes, variances and covariances
      * 'Updraft': updraft vertical profiles of some resolved and subgrid fluxes, variances and covariances
      * 'Downdraft': downdraft vertical profiles of some resolved and subgrid fluxes, variances and covariances
      * 'Radiation': radiative terms
      * 'Surface': averaged surface fields
      * 'Two\_point\_correlation'
      * 'Spectrum'
      * 'BU\_KE': total (resolved+subgrid) kinetic energy budget
      * 'BU\_THL2': temperature variance budget
      * 'BU\_WTHL': temperature flux budget
      * 'BU\_RT2': total water variance budget
      * 'BU\_WRT': total water flux budget
      * 'BU\_THLR': liquid potential temperature - total water covariance budget
      * 'BU\_SV2': total scalar variance budget (for scalar variables)
      * 'BU\_WSV': total water flux budget (for scalar variables)
      * 'Miscellaneous': miscellaneous terms (geometry, various unclassified averaged terms...)
*  *shape*: 'Cartesian' (not set, not used and merged onto group level if group='Two\_point\_correlation' or 'Spectrum')
*  *timeavg*: 'Time\_averaged' or 'Not\_time\_averaged'
*  *norm*: 'Normalized' or 'Not\_normalized'
*  *mask*: level enabled even if no masks are used, in that case it is set to 'cart'. Except if group='Two\_point\_correlation' or 'Spectrum' (in that case: not set, not used and merged into norm level).  Possible values:
      * 'cart' (no mask)
      * 'neb', 'clear' if LLES\_NEB\_MASK=TRUE
      * 'core', 'env' if LLES\_CORE\_MASK=TRUE
      * 'usernnn' (with $nnn$ the number of the mask) if LLES\_MY\_MASK=TRUE
      * 'cs1', 'cs2', 'cs3' if LLES\_CS\_MASK=TRUE

Remarques :

*  some of the dimensions might not appear depending on the option set in the NAM\_LES and NAM\_PDF namelists
*  some metadata is also available in the file and provide information about the boundaries of the cartesian box or masks, averaging in the different dimensions, normalization...

#### Flyers

##### Liste des dimensions

*  *time\_flyer*: sampling times (different for each flyer)

##### Description des niveaux (groupes HDF5)

*  *category*: 'Flyers'
*  *subcategory*: 'Aircrafts', 'Radiosonde\_balloons, 'Isodensity\_balloons', 'Constant\_volume\_balloons'
*  *group*: name of the flyer
*  *shape*: 'Point' or 'Vertical\_profile'
*  *timeavg*: 'Not\_time\_averaged' (not used, merged into shape level)
*  *norm*: 'Not\_normalized' (not used, merged into shape level)
*  *mask*: not set, not used, merged into shape level

In [27]:
grp=fl_dia['/Flyers']

grpname='Flyers'
file=fl_dia

for sgrpname in grp.groups:
    sgrpnamefull='/'+grpname+'/'+sgrpname
    print_group(file,sgrpnamefull)

Chemin groupe HDF5 :  /Flyers/Constant_volume_balloons
Groupe HDF5 :  Constant_volume_balloons
Attributs :
  levels                         = subcategory
  subcategory                    = Constant_volume_balloons
  comment                        = Level for the flyers of type: Constant_volume_balloons
Liste des dimensions : 
Sous-groupes:
   CVB1MOBI
   CVB2MOBI
Variables:
----------------------------------------------------------------------------------
Chemin groupe HDF5 :  /Flyers/Aircrafts
Groupe HDF5 :  Aircrafts
Attributs :
  levels                         = subcategory
  subcategory                    = Aircrafts
  comment                        = Level for the flyers of type: Aircrafts
Liste des dimensions : 
Sous-groupes:
   DIMO19A
Variables:
----------------------------------------------------------------------------------


In [28]:
grp=fl_dia['/Flyers/Constant_volume_balloons']

grpname='Constant_volume_balloons'
file=fl_dia['Flyers']

for sgrpname in grp.groups:
    sgrpnamefull='/'+grpname+'/'+sgrpname
    print_group(file,sgrpnamefull)

Chemin groupe HDF5 :  /Constant_volume_balloons/CVB1MOBI
Groupe HDF5 :  CVB1MOBI
Attributs :
  levels                         = group
  group                          = CVB1MOBI
  comment                        = Values for flyer CVB1MOBI
Liste des dimensions : 
   time_flyer
Sous-groupes:
   Point
   Vertical_profile
Variables:
   time_flyer
----------------------------------------------------------------------------------
Chemin groupe HDF5 :  /Constant_volume_balloons/CVB2MOBI
Groupe HDF5 :  CVB2MOBI
Attributs :
  levels                         = group
  group                          = CVB2MOBI
  comment                        = Values for flyer CVB2MOBI
Liste des dimensions : 
   time_flyer
Sous-groupes:
   Point
   Vertical_profile
Variables:
   time_flyer
----------------------------------------------------------------------------------


In [29]:
grp=fl_dia['/Flyers/Constant_volume_balloons/CVB1MOBI']

grpname='CVB1MOBI'
file=fl_dia['/Flyers/Constant_volume_balloons']

for sgrpname in grp.groups:
    sgrpnamefull='/'+grpname+'/'+sgrpname
    print_group(file,sgrpnamefull)

Chemin groupe HDF5 :  /CVB1MOBI/Point
Groupe HDF5 :  Point
Attributs :
  levels                         = shape timeavg norm mask
  shape                          = Point
  comment                        = Values at position of flyer CVB1MOBI
  moving                         = yes
  time_averaged                  = no
  normalized                     = no
  mask                           = 
Liste des dimensions : 
Sous-groupes:
Variables:
   ZS
   Z
   MODEL
   LON
   LAT
   ZON_WIND
   MER_WIND
   W
   Th
   Rv
   Rhod
   Tke
   H_FLUX
   LE_FLUX
   Tke_Diss
   Tsrad
   X
   Y
----------------------------------------------------------------------------------
Chemin groupe HDF5 :  /CVB1MOBI/Vertical_profile
Groupe HDF5 :  Vertical_profile
Attributs :
  levels                         = shape timeavg norm mask
  shape                          = Vertical_profile
  comment                        = Vertical profiles at position of flyer CVB1MOBI
  moving                         = yes
  time

In [30]:
print_field(fl_dia['/Flyers/Constant_volume_balloons/CVB1MOBI/Point'],'Tke')

Variable :  Tke
Type :  float64
Liste des dimensions : 
   time_flyer
Taille(s) : 
   81
Attributs :
  long_name            = Tke
  units                = m2 s-2
  comment              = Turbulent kinetic energy
  _FillValue           = 9.969209968386869e+36
  valid_min            = -1e+36
  valid_max            = 1e+36


#### Profilers

##### Liste des dimensions

*  *time\_profiler*: sampling times

##### Description des niveaux (groupes HDF5)

*  *category*: 'Profilers'
*  *subcategory*: not set, not used, merged into category level
*  *group*: name of the profiler
*  *shape*: 'Point' or 'Vertical\_profile'
*  *timeavg*: 'Not\_time\_averaged' (not used, merged into shape level)
*  *norm*: 'Not\_normalized' (not used, merged into shape level)

#### Stations

##### Liste des dimensions

*  *time\_station*: sampling times

##### Description des niveaux (groupes HDF5)

*  *category*: 'Stations'
*  *subcategory*: not set, not used, merged into category level
*  *group*: name of the station
*  *shape*: 'Point' (not used, merged into group level)
*  *timeavg*: 'Not\_time\_averaged' (not used, merged into group level)
*  *norm*: 'Not\_normalized' (not used, merged into group level)
*  *mask*: not set, not used, merged into group level

#### Time series

##### Liste des dimensions

*  *series\_level, series\_level\_w*: vertical levels (for mass- and w-points respectively)
*  *time\_series*: sampling times

##### Description des niveaux (groupes HDF5)

*  *category*: 'Time\_series'
*  *subcategory*: not set, not used, merged into category level
*  *group*: 'TSERIES', 'ZTSERIES', 'XTSERIESnn' (with $nn$) the number of y-slice, see NAM\_SERIESn namelist)
*  *shape*: 'Cartesian' (not used, merged into group level)
*  *timeavg*: 'Not\_time\_averaged' (not used, merged into group level)
*  *norm*: 'Not\_normalized' (not used, merged into group level)
*  *mask*: 'GLOB', 'LAND', 'SEA' (if group='XTSERIESnn': not set, not used, merged into group level)

## Namelistes

### NAM_BACKUP

*  Nameliste pour les fichiers de reprise (backups)
*  Remplace NAM_FMOUT depuis Méso-NH version 5.4.0

| Variable | Défaut} | Commentaire |
| :-       | :-      | :-          |
| XBAK_TIME(m,i)          | XNEGUNDEF | temps depuis début segment (en secondes) |
| NBAK_STEP(m,i)          | NNEGUNDEF | pas de temps depuis début segment |
| XBAK_TIME_FREQ(m)       | XNEGUNDEF | fréquence en temps (en secondes) |
| XBAK_TIME_FREQ_FIRST(m) | 0.        | instant premier backup |
| NBAK_STEP_FREQ(m)       | NNEGUNDEF | fréquence en pas de temps |
| NBAK_STEP_FREQ_FIRST(m) | 1         | pas de temps premier backup |
| LBAK_BEG                | .FALSE.   | force écriture après premier pas de temps |
| LBAK_END                | .FALSE.   | force écriture après dernier pas de temps |
| CBAK_DIR                | ''        | répertoire pour écrire les fichiers de reprise et diachroniques (prioritaire sur CIO_DIR) |

*m* pour le numéro du modèle, *i* pour l'instant

### NAM_OUTPUT

*  Nameliste pour les fichiers de sortie (outputs)

| Variable | Défaut} | Commentaire |
| :-       | :-      | :-          |
| COUT_VAR(m,p)                  | ''           | noms des champs à mettre dans output |
| XOUT_TIME(m,i)                 | XNEGUNDEF    | temps depuis début segment (en secondes) |
| NOUT_STEP(m,i)                 | NNEGUNDEF    | pas de temps depuis début segment |
| XOUT_TIME_FREQ(m)              | XNEGUNDEF    | fréquence en temps (en secondes) |
| XOUT_TIME_FREQ_FIRST(m)        | 0.           | instant première sortie |
| NOUT_STEP_FREQ(m)              | NNEGUNDEF    | fréquence en pas de temps |
| NOUT_STEP_FREQ_FIRST(m)        | 1            | pas de temps première sortie |
| LOUT_BEG                       | .FALSE.      | force écriture après premier pas de temps |
| LOUT_END                       | .FALSE.      | force écriture après dernier pas de temps |
| LOUT_REDUCE_FLOAT_PRECISION(m) | .FALSE.      | réduction précision des flottants (netCDF) |
| LOUT_COMPRESS(m)               | .FALSE.      | compression données (netCDF) |
| NOUT_COMPRESS_LEVEL(m)         |  4           | niveau compression (netCDF) |
| LOUT_COMPRESS_LOSSY(m)         | .FALSE.      | compression données avec pertes (netCDF, seulement tableaux de réels) |
| COUT_COMPRESS_LOSSY_ALGO(m)    | 'GRANULARBR' | algorithme de quantification ('BitGroom', 'GranularBR' ou 'BitRound') |
| NOUT_COMPRESS_LOSSY_NSD(m)     |  3           | nombre de décimales (ou bits) significatifs à conserver |
| COUT_DIR                       | ''           | répertoire pour les fichiers de sortie (prioritaire sur CIO\_DIR) |

*m* pour le numéro du modèle, *i* pour l'instant

**Attention :** tous les noms de champs ne sont pas (encore) référencés. Si pas disponible, utiliser IO\_WRITE\_FIELD\_USER ou me contacter.

###  NAM_CONFIO

*  Nameliste pour la configuration globale des entrées-sorties

| Variable | Défaut} | Commentaire |
| :-       | :-      | :-          |
| LCDF4               | .FALSE.  | active netCDF (forcé à true si *LLFIOUT=.FALSE.*) |
| LLFIOUT             | .FALSE.  | écrit les fichiers au format LFI (en plus de netCDF si *LCDF4=.TRUE.*) |
| LLFIREAD            | .FALSE.  | force la lecture des fichiers au format LFI |
| CIO_DIR             | ''       | répertoire pour les écritures (fichiers *backups*, diachroniques et *outputs*) |
| LVERB_OUTLST        | .TRUE.   | messages de diagnostiques dans fichiers *OUTPUT_LISTINGn* |
| LVERB_STDOUT        | .FALSE.  | messages de diagnostiques sur la sortie standard |
| LVERB_ALLPRC        | .FALSE.  | message de diagnostiques de tous les processus (seulement le premier si *.FALSE.*) |
| NGEN_VERB           | 4        | niveau de verbosité pour les messages diagnostiques 'génériques' |
| NGEN_ABORT_LEVEL    | 2        | niveau minimum requis de diagnostique 'générique" pour arrêt forcé de l'application |
| NIO_VERB            | 4        | niveau de verbosité pour les messages diagnostiques sur les I/O |
| NIO_ABORT_LEVEL     | 2        | niveau minimum requis de diagnostique I/O pour arrêt forcé de l'application |
| NBUD_VERB           | 4        | niveau de verbosité pour les messages diagnostiques sur les bilans/budgets |
| NBUD_ABORT_LEVEL    | 2        | niveau minimum requis de diagnostique bilans pour arrêt forcé de l'application |
| LIO_ALLOW_NO_BACKUP | .FALSE.  | autorise la possibilité de ne pas créer de fichiers de reprises (pour tests) |
| LIO_NO_WRITE        | .FALSE.  | désactive toutes les écritures de fichiers (pour tests) |

Remarque : si un fichier en lecture n'est pas trouvé dans le format demandé (netCDF ou LFI), Méso-NH tentera d'en trouver un dans l'autre format (extension du fichier)

Niveaux de verbosité :

| # | Mot clé       | Description     |
| :- | :-           | :-              |
| 0 | NVERB_NO      | pas de messages |
| 1 | NVERB_FATAL   | erreurs fatales |
| 2 | NVERB_ERROR   | erreurs         |
| 3 | NVERB_WARNING | avertissements  |
| 4 | NVERB_INFO    | informations    |
| 5 | NVERB_DEBUG   | débogage        |

### NAM_BU_*

Les namelistes *NAM_BU_R* * contiennent 2 entrées chacune (*nn* correspond à la variable pronostique) depuis la version 5.5.0. Auparavant, il y avait une entrée par terme source.

*  *LBU_nn* : pour activer ou non la collecte des termes sources du bilan
*  *CBULIST_nn* : tableau de chaînes de caractères pour spécifier les noms des termes sources à stocker
*  Plusieurs termes sources peuvent être combinés (additionnés) en les spécifiant dans la même entrée séparés par le symbole *+* (plus). Remarque : un terme source ne peut apparaître qu'à une seule position dans *CBULIST_nn*.
*  Il est possible de demander l'écriture de l'ensemble des termes sources en spécifiant le mot clé *ALL* en première position dans *CBULIST_nn*.

Exemple :
```fortran
&NAM_BU_RU
  LBU_RU = .TRUE.
  CBULIST_RU(1)='ADV'
  CBULIST_RU(2)='HTURB+VTURB' /
&NAM_BU_RV
  LBU_RV = .TRUE.
  CBULIST_RV(1)='ALL' /
```

Le choix des autres paramètres des bilans se font grâce à la nameliste *NAM_BUDGET* (pas présentée ici).

### NAM_CONFZ

*  Nameliste pour la configuration du découpage en plans verticaux
*  Cette nameliste contient aussi des variables non pertinentes pour les entrées-sorties (pas montrées ici)

| Variable | Défaut  | Commentaire |
| :-       | :-      | :-          |
| NB_PROCIO_R | 1 | nombre de fichiers et de processus utilisés pour le découpage en niveaux verticaux en lecture  |
| NB_PROCIO_W | 1 | nombre de fichiers et de processus utilisés pour le découpage en niveaux verticaux en écriture | 

<a id='code_source'></a>
## Code source

Dans cette partie, sont évoqués :
*  la [gestion des fichiers](#gestion_fichiers) (liste, ouverture, fermeture),
*  la [gestion des champs](#gestion_champs),
*  les [fichiers d'outputs](#src_fichiers_outputs),
*  les [fichiers diachroniques](#src_fichiers_diachro),
*  les [messages de diagnostique](#messages).

<a id="gestion_fichiers"></a>
### Gestion des fichiers

La gestion des fichiers se fait en 3 phases :
*  ajout à la liste des fichiers avec ses caractéristiques,
*  ouverture du fichier,
*  fermeture du fichier.

Remarque : actuellement, il n'est pas possible de retirer un fichier de la liste.

#### Ajout fichier

L'ajout d'un fichier à la liste des fichiers se fait via un appel à *IO_File_add2list*.
Cette étape est nécessaire avant de pouvoir l'ouvrir.

```fortran
SUBROUTINE IO_File_add2list(TPFILE,HNAME,HTYPE,HMODE,                 &
                            HFORM,HACCESS,HFORMAT,HDIRNAME,           &
                            KLFINPRAR,KLFITYPE,KLFIVERB,KRECL,KMODEL, &
                            TPDADFILE,TPDATAFILE,OOLD,OSPLIT_IOZ)

TYPE(TFILEDATA),POINTER,         INTENT(INOUT) :: TPFILE    !File structure to return
CHARACTER(LEN=*),                INTENT(IN)    :: HNAME     !Filename
CHARACTER(LEN=*),                INTENT(IN)    :: HTYPE     !Filetype (backup, output, prepidealcase...)
CHARACTER(LEN=*),                INTENT(IN)    :: HMODE     !Opening mode (read, write...)
CHARACTER(LEN=*),       OPTIONAL,INTENT(IN)    :: HFORM     !Formatted/unformatted
CHARACTER(LEN=*),       OPTIONAL,INTENT(IN)    :: HACCESS   !Direct/sequential/stream
CHARACTER(LEN=*),       OPTIONAL,INTENT(IN)    :: HFORMAT   !Fileformat (NETCDF4, LFI, LFICDF4...)
CHARACTER(LEN=*),       OPTIONAL,INTENT(IN)    :: HDIRNAME  !File directory
INTEGER(KIND=LFIINT),   OPTIONAL,INTENT(IN)    :: KLFINPRAR !Number of predicted articles of the LFI file (non crucial)
INTEGER,                OPTIONAL,INTENT(IN)    :: KLFITYPE  !Type of the file (used to generate list of files to transfers)
INTEGER,                OPTIONAL,INTENT(IN)    :: KLFIVERB  !LFI verbosity level
INTEGER,                OPTIONAL,INTENT(IN)    :: KRECL     !Record length
INTEGER,                OPTIONAL,INTENT(IN)    :: KMODEL    !Model number
TYPE(TFILEDATA),POINTER,OPTIONAL,INTENT(IN)    :: TPDADFILE !Corresponding dad file
TYPE(TFILEDATA),POINTER,OPTIONAL,INTENT(IN)    :: TPDATAFILE!Corresponding data file (used only for DES files)
LOGICAL,                OPTIONAL,INTENT(IN)    :: OOLD      !FALSE if new file (should not be found)
                                                            !TRUE if the file could already be in the list
                                                            !     (add it only if not yet present)
logical,                optional,intent(in)    :: osplit_ioz !Is the file split vertically
```

Liste des types de fichiers autorisés (argument *HTYPE*) :
*  CHEMINPUT
*  CHEMTAB
*  DES
*  GPS
*  METEO
*  NML
*  OUTPUTLISTING
*  SURFACE_DATA
*  TXT
*  MNH
*  MNHBACKUP
*  MNHDIACHRONIC
*  MNHDIAG 
*  MNHOUTPUT
*  PGD

Liste des modes d'accès autorisés (argument *HMODE*) :
*  READ
*  WRITE

Selon le type de fichiers, certains modes d'accès ne sont pas possibles. Par exemple, les fichiers de type nameliste (*NML*) ne peuvent être ouverts qu'en lecture.

Exemple :
```fortran
USE MODD_IO, ONLY: TFILEDATA
USE MODE_IO_MANAGE_STRUCT, only: IO_File_add2list

TYPE(TFILEDATA), POINTER :: TZFILE

TZFILE => NULL()
CALL IO_File_add2list( TZFILE, 'nom_de_mon_fichier', 'MNH', 'READ' )
```

#### Ouverture fichier

Une fois un fichier intégré dans la liste des fichiers, celui-ci peut être ouvert.

```fortran
recursive SUBROUTINE IO_File_open(TPFILE,KRESP,kmasterrank, HPOSITION,HSTATUS,HPROGRAM_ORIG)

TYPE(TFILEDATA),  POINTER,  INTENT(INOUT) :: TPFILE ! File structure
INTEGER,          optional, INTENT(OUT)   :: KRESP  ! Return code
integer,          optional, intent(in)    :: kmasterrank !Rank of the master process
CHARACTER(LEN=*), optional, INTENT(IN)    :: HPOSITION
CHARACTER(LEN=*), optional, INTENT(IN)    :: HSTATUS
CHARACTER(LEN=*), optional, INTENT(IN)    :: HPROGRAM_ORIG !To emulate a file coming from this program
```

Exemple :
```fortran
USE MODE_IO_FILE, only: IO_File_open
CALL IO_File_open(TZFILE)
```

#### Fermeture fichier

Lorsque l'on n'a plus besoin d'accéder à un fichier, il peut être fermé. Cette étape bien que non obligatoire est fortement conseillée (surtout en écriture pour éviter les pertes de données), mais aussi pour libérer des ressources.

```fortran
recursive SUBROUTINE IO_File_close(TPFILE,KRESP,HPROGRAM_ORIG)

TYPE(TFILEDATA),            INTENT(INOUT) :: TPFILE ! File structure
INTEGER,          OPTIONAL, INTENT(OUT)   :: KRESP  ! Return code
CHARACTER(LEN=*), OPTIONAL, INTENT(IN)    :: HPROGRAM_ORIG !To emulate a file coming from this program
```

Exemple :
```fortran
USE MODE_IO_FILE, only: IO_File_close
CALL IO_File_close(TZFILE)
```

#### Structure de données *TFILEDATA*

Pour information, la structure de données passée en premier argument pour toutes les manipulations de fichiers ressemble à ceci :

```fortran
!Structure describing the characteristics of a file
TYPE TFILEDATA
  CHARACTER(LEN=NFILENAMELGTMAX) :: CNAME = '' !Filename
  CHARACTER(LEN=:),ALLOCATABLE   :: CDIRNAME   !Directory name
  CHARACTER(LEN=13) :: CTYPE   = "UNKNOWN" !Filetype (PGD, MNH, DES, NML...)
  CHARACTER(LEN=7)  :: CFORMAT = "UNKNOWN" !Fileformat (NETCDF4, LFI, LFICDF4...)
  CHARACTER(LEN=7)  :: CMODE   = "UNKNOWN" !Opening mode (read, write...)
  LOGICAL           :: LOPENED = .FALSE.   !Is the file opened
  INTEGER           :: NOPEN_CURRENT = 0   !Number of times the file is currently opened (several opens without close are allowed)
  INTEGER           :: NOPEN   = 0         !Number of times the file has been opened (during the current execution)
  INTEGER           :: NCLOSE  = 0         !Number of times the file has been closed (during the current execution)
  !
  INTEGER           :: NMASTER_RANK  = -1      !Rank of the master process (no meaning if LMULTIMASTERS=.T.)
  INTEGER           :: NMPICOMM      = -1      !MPI communicator used for IO on this file
  LOGICAL           :: LMASTER       = .FALSE. !True if process is master of the file (process that open/read/write/close)
  LOGICAL           :: LMULTIMASTERS = .FALSE. !True if several processes may access the file
#if ( MNH_REDUCE_DIMENSIONS_IN_FILES == 1 )
  logical           :: ldimreduced   = .true.  !True if number of dimensions of fields can be reduced (for 2D simulations)
#else
  logical           :: ldimreduced   = .false. !True if number of dimensions of fields can be reduced (for 2D simulations)
#endif
  !
  INTEGER           :: NSUBFILES_IOZ = 0       !Number of sub-files (Z-split files based on this file)
                                               !For example if 2 sub-files and this file is abcd,
                                               !the 2 sub-files are abcd.Z001 and abcd.Z002
  TYPE(TFILE_ELT),DIMENSION(:),ALLOCATABLE :: TFILES_IOZ !Corresponding Z-split files
  !
  INTEGER              :: NMODEL = 0              !Model number corresponding to the file (field not always set)
  INTEGER,DIMENSION(3) :: NMNHVERSION = (/0,0,0/) !MesoNH version used to create the file
  !
#ifdef MNH_IOLFI
  ! Fields for LFI files
  INTEGER(KIND=LFIINT) :: NLFININAR = 0  !Number of articles of the LFI file (only accurate if file opened in read mode)
  INTEGER(KIND=LFIINT) :: NLFINPRAR = 0  !Number of predicted articles of the LFI file (non crucial)
  INTEGER              :: NLFITYPE  = -1 !Type of the file (used to generate list of files to transfers)
  INTEGER              :: NLFIVERB  = 1  !LFI verbosity level
  INTEGER(KIND=LFIINT) :: NLFIFLU   = -1 !File identifier
#endif
  !
#ifdef MNH_IOCDF4
  ! Fields for netCDF files
  INTEGER(KIND=CDFINT)   :: NNCID = -1 !File identifier (corresponding to the actual group)
  INTEGER(KIND=CDFINT)   :: NNCNAR = 0 !Number of articles of the netCDF file (only accurate if file opened in read mode)
  LOGICAL                :: LNCREDUCE_FLOAT_PRECISION = .FALSE. ! Reduce the precision of floats to single precision
                                                                ! instead of double precision
  LOGICAL                :: LNCCOMPRESS = .FALSE. ! Do compression on fields
  INTEGER(KIND=CDFINT)   :: NNCCOMPRESS_LEVEL = 0 ! Compression level
  type(tdimsnc), pointer :: tncdims => Null()     ! Dimensions of netCDF file
#endif
  !
  !Fields for other files
  INTEGER :: NLU = -1                      !Logical unit number
  INTEGER :: NRECL = -1                    !Fortran RECL (record length)
  CHARACTER(LEN=11) :: CFORM   = "UNKNOWN" !Fortran FORM (FORMATTED/UNFORMATTED)
  CHARACTER(LEN=10) :: CACCESS = "UNKNOWN" !Fortran ACCESS (DIRECT/SEQUENTIAL/STREAM)
  !
  TYPE(TFILEDATA),POINTER :: TDADFILE   => NULL() !Corresponding dad file
  TYPE(TFILEDATA),POINTER :: TDESFILE   => NULL() !Corresponding .des file
  TYPE(TFILEDATA),POINTER :: TDATAFILE  => NULL() !Corresponding data file (if .des file)
  TYPE(TFILEDATA),POINTER :: TMAINFILE  => NULL() !Corresponding main file if the file is an sub-file
  !
  TYPE(TFILEDATA),POINTER :: TFILE_PREV => NULL()
  TYPE(TFILEDATA),POINTER :: TFILE_NEXT => NULL()
END TYPE TFILEDATA

!Structure containing one pointer to a file
!Useful to create arrays of pointers to files
TYPE TFILE_ELT
  TYPE(TFILEDATA),POINTER :: TFILE => NULL()
END TYPE TFILE_ELT

```

<a id="gestion_champs"></a>
### Gestion des champs

#### Structure de données *TFIELDMETADATA*

Avant Méso-NH 5.6.0, les métadonnées étaient définies dans la structure de donnée *TFIELDDATA*.

Avant de réaliser des opérations de lecture ou d'écriture, un champ (ou variable) doit avoir des métadonnées associées. Les métadonnées sont des informations sur les caractéristiques de celui-ci.

Métadonnées principales :

| Nom         | CF  | Description |
| :-          | :-  | :-          |
| *CMNHNAME*  | non | nom du champ  |
| *CSTDNAME*  | oui | nom standard (doit être dans la [liste officielle des conventions CF](http://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html)) |
| *CLONGNAME* | oui | nom long (description courte) |
| *CUNITS*    | oui | unités |
| *CCOMMENT*  | non | commentaires |
| *NGRID*     | non | numéro de grille Arakawa (mettre 0 si n'a pas de sens) |
| *NTYPE*     | non | type des données (TYPEINT, TYPELOG, TYPEREAL, TYPECHAR ou TYPEDATE) |
| *NDIMS*     | oui | nombre de dimensions |
| *NDIMLIST*  | oui | liste des dimensions |
| *CDIR*      | non | direction de distribution des données (--, XX, YY, ZZ ou XY) |
| *LTIMEDEP*  | non | vrai si la variable change au cours du temps |

Remarques :
*  La colonne CF correspond à l'existence de métadonnées dans les [conventions CF](http://cfconventions.org).
*  Si le champ n'a pas de [nom standard dans les conventions CF](http://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html), le laisser vide mais ne surtout rien y mettre.
*  Les unités doivent être compatibles avec [UDUNITS](http://www.unidata.ucar.edu/software/udunits/). Pour des valeurs adimensionnelles, mettre 1 et, si nécessaire, ajouter des informations complémentaires dans *CLONGNAME*.
*  Si la liste des dimensions n'est pas fournie, Méso-NH tente de les deviner en fonction du numéro de grille, du nombre de dimensions et de la valeur de *CDIR*. Pour des champs 2D horizontaux ou 3D sur l'ensemble du domaine, cela fonctionne parfaitement.
*  *CDIR* permet de stocker une information sur la manière dont les données sont distribuées sur les processus MPI de Méso-NH. Les plus courantes sont *--* (ou valeur vide) pour des données similaires sur tous les processus et *XY* pour des données 2D ou 3D distribuées sur l'ensemble des processus (tous les champs distribués sur les grilles des modèles).
*  Toutes les métadonnées ne doivent pas obligatoirement être précisées. Seuls le nom du champ, le numéro de grille et le type de données doivent toujours être donnés. Selon les cas, les valeurs par défaut des autres métadonnées peuvent être suffisantes. Idéalement, il vaut mieux fournir le plus d'informations possible.

Code source (Méso-NH 5.6.1) :
```Fortran
type :: tfieldmetadata_base
  CHARACTER(LEN=NMNHNAMELGTMAX)  :: CMNHNAME  = '' !Name of the field (for MesoNH, non CF convention)
  CHARACTER(LEN=NSTDNAMELGTMAX)  :: CSTDNAME  = '' !Standard name (CF convention)
  CHARACTER(LEN=NLONGNAMELGTMAX) :: CLONGNAME = '' !Long name (CF convention)
  CHARACTER(LEN=NUNITLGTMAX)     :: CUNITS    = '' !Canonical units (CF convention)
  CHARACTER(LEN=NCOMMENTLGTMAX)  :: CCOMMENT  = '' !Comment (for MesoNH, non CF convention)
!   CHARACTER(LEN=:),allocatable :: CCOMMENT  !Comment (for MesoNH, non CF convention)
  INTEGER            :: NGRID     = NGRIDUNKNOWN !Localization on the model grid
  INTEGER            :: NTYPE     = TYPEUNDEF !Datatype
  INTEGER            :: NDIMS     = 0  !Number of dimensions
  INTEGER, DIMENSION(NMNHMAXDIMS) :: NDIMLIST = NMNHDIM_UNKNOWN ! List of dimensions of the data field
  !
#ifdef MNH_IOCDF4
  INTEGER            :: NFILLVALUE =  NF90_FILL_INT  !Fill value for integer fields
  REAL               :: XFILLVALUE =  NF90_FILL_REAL !Fill value for real fields
                                                     !NF90_FILL_REAL is the default fill value
                                                     !used by netCDF to pre-fill real and also double
                                                     !variables
#else
  INTEGER            :: NFILLVALUE =  -2147483647            !Fill value for integer fields
  REAL               :: XFILLVALUE =  9.9692099683868690e+36 !Fill value for real fields
#endif
  INTEGER            :: NVALIDMIN  = -2147483646 !Minimum valid value for integer fields
  INTEGER            :: NVALIDMAX  =  2147483647 !Maximum valid value for integer fields
  REAL               :: XVALIDMIN  = -1.E36 !Minimum valid value for real fields
  REAL               :: XVALIDMAX  =  1.E36 !Maximum valid value for real fields
end type tfieldmetadata_base

TYPE, extends( tfieldmetadata_base ) :: TFIELDMETADATA
  CHARACTER(LEN=2)   :: CDIR      = '' !Type of the data field (XX,XY,--...)
  CHARACTER(LEN=4)   :: CLBTYPE   = 'NONE' !Type of the lateral boundary (LBX,LBY,LBXU,LBYV)
  LOGICAL            :: LTIMEDEP  = .FALSE. !Is the field time-dependent?
END TYPE TFIELDMETADATA

!Structure describing the characteristics of a field
TYPE, EXTENDS( TFIELDMETADATA ) :: TFIELDDATA
  INTEGER :: NMODELMAX = -1 !Number of models for which the field has been allocated (default value must be negative)
  !
  TYPE(TFIELDPTR_C0D),DIMENSION(:),ALLOCATABLE :: TFIELD_C0D !Pointer to the character string fields (one per nested mesh)
  TYPE(TFIELDPTR_C1D),DIMENSION(:),ALLOCATABLE :: TFIELD_C1D !Pointer to the character string 1D fields (one per nested mesh)
  !
  TYPE(TFIELDPTR_L0D),DIMENSION(:),ALLOCATABLE :: TFIELD_L0D !Pointer to the scalar logical fields (one per nested mesh)
  TYPE(TFIELDPTR_L1D),DIMENSION(:),ALLOCATABLE :: TFIELD_L1D !Pointer to the logical 1D fields (one per nested mesh)
  !
  TYPE(TFIELDPTR_N0D),DIMENSION(:),ALLOCATABLE :: TFIELD_N0D !Pointer to the scalar integer fields (one per nested mesh)
  TYPE(TFIELDPTR_N1D),DIMENSION(:),ALLOCATABLE :: TFIELD_N1D !Pointer to the integer 1D fields (one per nested mesh)
  TYPE(TFIELDPTR_N2D),DIMENSION(:),ALLOCATABLE :: TFIELD_N2D !Pointer to the integer 2D fields (one per nested mesh)
  TYPE(TFIELDPTR_N3D),DIMENSION(:),ALLOCATABLE :: TFIELD_N3D !Pointer to the integer 3D fields (one per nested mesh)
  !
  TYPE(TFIELDPTR_X0D),DIMENSION(:),ALLOCATABLE :: TFIELD_X0D !Pointer to the scalar real fields (one per nested mesh)
  TYPE(TFIELDPTR_X1D),DIMENSION(:),ALLOCATABLE :: TFIELD_X1D !Pointer to the real 1D fields (one per nested mesh)
  TYPE(TFIELDPTR_X2D),DIMENSION(:),ALLOCATABLE :: TFIELD_X2D !Pointer to the real 2D fields (one per nested mesh)
  TYPE(TFIELDPTR_X3D),DIMENSION(:),ALLOCATABLE :: TFIELD_X3D !Pointer to the real 3D fields (one per nested mesh)
  TYPE(TFIELDPTR_X4D),DIMENSION(:),ALLOCATABLE :: TFIELD_X4D !Pointer to the real 4D fields (one per nested mesh)
  TYPE(TFIELDPTR_X5D),DIMENSION(:),ALLOCATABLE :: TFIELD_X5D !Pointer to the real 5D fields (one per nested mesh)
  TYPE(TFIELDPTR_X6D),DIMENSION(:),ALLOCATABLE :: TFIELD_X6D !Pointer to the real 6D fields (one per nested mesh)
  !
  TYPE(TFIELDPTR_T0D),DIMENSION(:),ALLOCATABLE :: TFIELD_T0D !Pointer to the scalar date/time fields (one per nested mesh)
  TYPE(TFIELDPTR_T1D),DIMENSION(:),ALLOCATABLE :: TFIELD_T1D !Pointer to the date/time 1D fields (one per nested mesh)
END TYPE TFIELDDATA
```

#### *TFIELDLIST*

Une partie des champs de Méso-NH est accessible via la variable *TFIELDLIST* (accessible via le module *MODD_FIELD*).

*  Il s'agit d'un tableau d'éléments de type *TFIELDDATA*.
*  Il contient les métadonnées d'une série de champs,
*  ainsi que des pointeurs vers leur contenu.
*  *TFIELDLIST* est actuellement complétée dans la procédure *INI_FIELD_LIST* (*LIB/SURCOUCHE/src/mode_field.f90*)



##### A partir de Méso-NH 5.6.0

Exemple pour le champ *XUT* (Méso-NH 5.6.1) :
```fortran
SUBROUTINE INI_FIELD_LIST()

call Add_field2list( TFIELDDATA(            &
  CMNHNAME   = 'UT',                        &
  CSTDNAME   = 'x_wind',                    &
  CLONGNAME  = 'UT',                        &
  CUNITS     = 'm s-1',                     &
  CDIR       = 'XY',                        &
  CCOMMENT   = 'X_Y_Z_U component of wind', &
  NGRID      = 2,                           &
  NTYPE      = TYPEREAL,                    &
  NDIMS      = 3,                           &
  LTIMEDEP   = .TRUE.                       ) )
```

* *Add_field2list* est une procédure qui gère *TFIELDLIST* en l'agrandissant si ncécessaire (par réallocation) qui prend pour seul argument une structure de type *TFIELDDATA*
* *TFIELDDATA(...)* est un constructeur personnalisé (*custom constructor* au sens Fortran 2003). Il s'utilise comme une *subroutine* portant le même nom que la structure de données qu'il traite. La liste des arguments autorisés correspond à tous les champs de cette structure. Dans le cas présent, le seul qui soit obligatoire est le type *NTYPE*. Les autres champs sont tous optionnels et ont des valeurs par défaut ou sont positionnées en fonction des autres informations fournies au constructeur. Néanmoins, il est conseillé d'en fournir le plus possible pour éviter les erreurs.

##### Avant Méso-NH 5.6.0

Exemple pour le champ *XUT* (Méso-NH 5.5.0) :
```fortran
SUBROUTINE INI_FIELD_LIST(KMODEL)

INTEGER,INTENT(IN),OPTIONAL :: KMODEL

TFIELDLIST(IDX)%CMNHNAME   = 'UT'
TFIELDLIST(IDX)%CSTDNAME   = 'x_wind'
TFIELDLIST(IDX)%CLONGNAME  = 'UT'
TFIELDLIST(IDX)%CUNITS     = 'm s-1'
TFIELDLIST(IDX)%CDIR       = 'XY'
TFIELDLIST(IDX)%CCOMMENT   = 'X_Y_Z_U component of wind'
TFIELDLIST(IDX)%NGRID      = 2
TFIELDLIST(IDX)%NTYPE      = TYPEREAL
TFIELDLIST(IDX)%NDIMS      = 3
TFIELDLIST(IDX)%LTIMEDEP   = .TRUE.
ALLOCATE(TFIELDLIST(IDX)%TFIELD_X3D(IMODEL))
```

#### Changement de modèle (*grid nesting*)

*  Si le champ dépend du modèle imbriqué, il faut mettre à jour le pointeur vers les bonnes données lors des changements de modèles (appelé lors des appels à *GOTO_MODEL*).
*  La mise à jour se fait dans *FIELDLIST_GOTO_MODEL*.

Exemple pour le champ *XUT* (à partir de Méso-NH 5.6.0) :
```fortran
SUBROUTINE FIELDLIST_GOTO_MODEL(KFROM, KTO)

INTEGER, INTENT(IN) :: KFROM, KTO

call Goto_model_1field( 'UT', kfrom, kto, xut )
```

Exemple pour le champ *XUT* (Méso-NH 5.5.0, déclaré dans le fichier *modd_fieldn.f90*) :
```fortran
SUBROUTINE FIELDLIST_GOTO_MODEL(KFROM, KTO)

INTEGER, INTENT(IN) :: KFROM, KTO

CALL FIND_FIELD_ID_FROM_MNHNAME('UT',IID,IRESP); TFIELDLIST(IID)%TFIELD_X3D(KFROM)%DATA => XUT

IF( KFROM/=KTO) THEN
CALL FIND_FIELD_ID_FROM_MNHNAME('UT',IID,IRESP); XUT => TFIELDLIST(IID)%TFIELD_X3D(KTO)%DATA
END IF
```

*  Une variable présente dans la liste peut être recherchée à partir de son nom

```fortran
USE MODE_FIELD, only: Find_field_id_from_mnhname

CALL FIND_FIELD_ID_FROM_MNHNAME( 'UT', IID, IRESP )
```

```fortran
SUBROUTINE FIND_FIELD_ID_FROM_MNHNAME(HMNHNAME,KID,KRESP,ONOWARNING)
!
CHARACTER(LEN=*),            INTENT(IN) :: HMNHNAME !Name of the field to find
INTEGER,                     INTENT(OUT):: KID      !Index of the field
INTEGER,                     INTENT(OUT):: KRESP    !Return-code 
LOGICAL, OPTIONAL,           INTENT(IN) :: ONOWARNING !If true, do not print warning
```

* Les champs décrits dans *TFIELDLIST* sont ceux qui peuvent être utilisés dans les fichiers d'*outputs* (sorties fréquentes) via la variable *COUT_VAR* de la nameliste *NAM_OUTPUT*.

#### Lecture / écriture d'un champ

La lecture ou l'écriture d'un champ peut se faire de 2 façons différentes :
* en donnant le nom de la variable à la condition que celle-ci soit bien définie dans la liste *TFIELDLIST*
* en fournissant les métadonnées dans une structure de type *TFIELDDATA*

##### Ecriture

En-têtes d'une sélection de sous-programmes d'écriture de champs :

```Fortran
SUBROUTINE IO_Field_write_byname_X0(TPFILE,HNAME,PFIELD,KRESP)
  TYPE(TFILEDATA),           INTENT(IN) :: TPFILE
  CHARACTER(LEN=*),          INTENT(IN) :: HNAME    ! name of the field to write
  REAL,                      INTENT(IN) :: PFIELD   ! array containing the data field
  INTEGER,OPTIONAL,          INTENT(OUT):: KRESP    ! return-code

SUBROUTINE IO_Field_write_byfield_X0(TPFILE,TPFIELD,PFIELD,KRESP)
  TYPE(TFILEDATA),             INTENT(IN) :: TPFILE
  TYPE(TFIELDDATA),            INTENT(IN) :: TPFIELD
  REAL,TARGET,                 INTENT(IN) :: PFIELD   ! array containing the data field
  INTEGER,OPTIONAL,            INTENT(OUT):: KRESP    ! return-code
```

Il existe différentes subroutines d'écritures qui acceptent soit un nom de champ, soit une structure *TFIELDDATA*. Il y en a pour différents types de données de base (entiers, flottants, logiques, caractères et dates) et différents nombre de dimensions de tableaux.

Elles sont toutes accessibles via un bloc interface *familial* de nom *IO_Field_write*.

Exemples :
```Fortran
USE MODD_FIELD,          only: TFIELDDATA
USE MODE_IO_FIELD_WRITE, only: IO_Field_write

TYPE(TFIELDDATA) :: TZFIELD

CALL IO_Field_write( TPFILE, 'UT', XUT )

TZFIELD = TFIELDMETADATA(                                     &
  CMNHNAME   = 'URECYCLN',                                    &
  CLONGNAME  = 'URECYCLN',                                    &
  CSTDNAME   = '',                                            &
  CUNITS     = 'm s-1',                                       &
  CDIR       = 'XY',                                          &
  NGRID      = 2,                                             &
  NTYPE      = TYPEREAL,                                      &
  NDIMS      = 3,                                             &
  NDIMLIST   = [ NMNHDIM_NI, NMNHDIM_NJ, NMNHDIM_NOTLISTED ], &
  LTIMEDEP   = .TRUE.,                                        &
  CCOMMENT   = 'UMEAN-NORTH side plan for recycling purpose'  )
  CALL IO_Field_write(TPFILE,TZFIELD,XUMEANN(:,:,:))
```

* *TFIELDMETADATA(...)* est un constructeur personnalisé (*custom constructor* au sens Fortran 2003). Il s'utilise comme une *subroutine* portant le même nom que la structure de données qu'il traite. La liste des arguments autorisés correspond à tous les champs de cette structure. Dans le cas présent, le seul qui soit obligatoire est le type NTYPE. Les autres champs sont tous optionnels et ont des valeurs par défaut ou sont positionnées en fonction des autres informations fournies au constructeur. Néanmoins, il est conseillé d'en fournir le plus possible pour éviter les erreurs.

#### Lecture

En-têtes d'une sélection de sous-programmes de lecture de champs :

```Fortran
SUBROUTINE IO_Field_read_byname_X0(TPFILE,HNAME,PFIELD,KRESP)
  TYPE(TFILEDATA),  INTENT(IN)    :: TPFILE
  CHARACTER(LEN=*), INTENT(IN)    :: HNAME    ! name of the field to write
  REAL,             INTENT(INOUT) :: PFIELD   ! data field
  INTEGER,OPTIONAL, INTENT(OUT)   :: KRESP    ! return-code

SUBROUTINE IO_Field_read_byfield_X0(TPFILE,TPFIELD,PFIELD,KRESP)
  TYPE(TFILEDATA),  INTENT(IN)    :: TPFILE
  TYPE(TFIELDDATA), INTENT(INOUT) :: TPFIELD
  REAL,             INTENT(INOUT) :: PFIELD   ! data field
  INTEGER,OPTIONAL, INTENT(OUT)   :: KRESP    ! return-code
```

Il existe différentes subroutines de lectures qui acceptent soit un nom de champ, soit une structure *TFIELDDATA*. Il y en a pour différents types de données de base (entiers, flottants, logiques, caractères et dates) et différents nombre de dimensions de tableaux.

Elles sont toutes accessibles via un bloc interface *familial* de nom *IO_Field_read*.

Exemples :
```Fortran
USE MODD_FIELD,          only: TFIELDDATA
USE MODE_IO_FIELD_WRITE, only: IO_Field_read

TYPE(TFIELDDATA) :: TZFIELD

CALL IO_Field_read( TPFILE, 'UT', XUT )

TZFIELD = TFIELDMETADATA(                                     &
  CMNHNAME   = 'URECYCLN',                                    &
  CLONGNAME  = 'URECYCLN',                                    &
  CSTDNAME   = '',                                            &
  CUNITS     = 'm s-1',                                       &
  CDIR       = 'XY',                                          &
  NGRID      = 2,                                             &
  NTYPE      = TYPEREAL,                                      &
  NDIMS      = 3,                                             &
  NDIMLIST   = [ NMNHDIM_NI, NMNHDIM_NJ, NMNHDIM_NOTLISTED ], &
  LTIMEDEP   = .TRUE.,                                        &
  CCOMMENT   = 'UMEAN-NORTH side plan for recycling purpose'  )
CALL IO_Field_read(TPINIFILE,TZFIELD,PUMEANN)
```

Lors de la lecture d'un champ, certaines vérifications sont faites notamment sur ses dimensions qui doivent correspondre à celle des données dans le fichier, mais également sur les métadonnées qui doivent être cohérentes.

En cas de différence dans les métadonnées, des messages d'avertissement voire d'erreurs peuvent être affichés. Ils peuvent être gérés via la nameliste *NAM_CONFIO* (paramètres *NIO_VERB* et *NIO_ABORT_LEVEL*).

<a id="src_fichiers_outputs"></a>
### Fichiers d'outputs (sorties fréquentes)

*  Permettent de n'écrire qu'une sélection de champs
*  Adaptés à des sorties fréquentes
*  Ne peuvent servir à un redémarrage de Méso-NH
*  Les données peuvent être compressées (sans pertes). Cela peut avoir un impact sur les performances (négatif pour l'étape de compression, positif pour la réduction de la quantité de données à écrire).
*  La précision des variables flottantes peut être réduite en simple précision (stockage sur 32 bits). Cela donne un gain d'un facteur 2 en espace disque par rapport à la double précision (qui est généralement le défaut avec MésoNH).

La sélection de variables à écrire se fait de 2 façons différentes :

*  utilisation de champs présents dans *TFIELDLIST*
*  champs utilisateurs (sans passer par *TFIELDLIST*)

#### Champs présents dans *TFIELDLIST*

Les noms de champs connus de *TFIELDLIST* doivent simplement être donnés dans la nameliste *NAM_OUTPUT* avec le paramètre *COUT_VAR*.
L'écriture est lancée dans Méso-NH par un appel à *IO_Fieldlist_write*.

Exemple :
```fortran
&NAM_OUTPUT
    COUT_VAR(1,1)='UT'
    COUT_VAR(1,2)='INPRR'
    XOUT_TIME_FREQ(1) = 60.
    LOUT_REDUCE_FLOAT_PRECISION = .TRUE.
    LOUT_COMPRESS = .TRUE.
```

#### Champs *utilisateurs* (non présents dans *TFIELDLIST*)

Pour les champs inexistants dans *TFIELDLIST*, il est nécessaire de passer par le sous-programme *IO_Field_user_write* (fichier *LIB/SURCOUCHE/src/mode_io_field_write.f90*).

Celui-ci contient quelques exemples. Pour ajouter un champ,
*  il suffit de compléter ses métadonnées,
*  de rendre accessible les données à écrire (via l'utilisation d'un module ou par son calcul localement dans le sous-programme)
*  et de faire un appel à *IO_Field_write*.

```fortran
SUBROUTINE IO_Field_user_write(TPOUTPUT)
!
USE MODD_FIELD_n,    ONLY: XUT
!
IMPLICIT NONE
!
TYPE(TOUTBAK),    INTENT(IN)  :: TPOUTPUT !Output structure
!
TYPE(TFIELDDATA) :: TZFIELD
!
INTEGER          :: IKB
!
IKB=JPVEXT+1
!
TZFIELD = TFIELDMETADATA( &
  CMNHNAME   = 'UTLOW',   &
  CLONGNAME  = '',        &
  CSTDNAME   = 'x_wind',  &
  CUNITS     = 'm s-1',   &
  CDIR       = 'XY',      &
  CCOMMENT   = 'X-component of wind at lowest physical level', &
  NGRID      = 2,         &
  NTYPE      = TYPEREAL,  &
  NDIMS      = 2,         &
  LTIMEDEP   = .TRUE.     )
CALL IO_Field_write( TPOUTPUT%TFILE, TZFIELD, XUT(:,:,IKB) )
!
END SUBROUTINE IO_Field_user_write
```

<a id='src_fichiers_diachro'></a>
### Fichiers diachroniques

Les fichiers diachroniques sont structurés en utilisant les groupes HDF5 appelés dans Méso-NH niveaux (*levels*).

La modification des sources par un utilisateur se fait généralement par l'ajout ou la modification de champs pour les sorties. Dans de rares cas, il faudra peut-être également faire évoluer le découpage en niveaux. Pour ces derniers, on ne montre ici que la structure de base.

La structuration en niveaux (*levels*) se fait via l'utilisation de la structure de données *tbudiachrometadata* :

```fortran
type :: tbudiachrometadata
  character(len=NBUNAMELGTMAX),  dimension(NMAXLEVELS) :: clevels  = '' !Name of the different groups/levels in the netCDF file
  character(len=NCOMMENTLGTMAX), dimension(NMAXLEVELS) :: ccomments ='' !Comments for the different groups/levels in the netCDF file
  character(len=1)              :: cdirection   = ''                    !Used for 2pt correlation and spectrum
  logical :: lmobile    = .false.                                       !Is the domain moving? (ie for aircrafts and balloons)
  logical :: licompress = .false.
  logical :: ljcompress = .false.
  logical :: lkcompress = .false.
  logical :: ltcompress = .false. ! true if values are time averaged (can be on multiple time periods)
  logical :: lnorm      = .false. ! true if values are normalized
  logical, dimension(NMAXLEVELS) :: lleveluse = .false.
  integer :: nil = -1 !Cartesian box boundaries in physical domain coordinates
  integer :: nih = -1
  integer :: njl = -1
  integer :: njh = -1
  integer :: nkl = -1
  integer :: nkh = -1
  integer :: nsv = -1 !Reference number of the corresponding scalar variable
end type tbudiachrometadata
```

Les différents niveaux peuvent être référencés en passant par des *parameters* (voir dans *MNH/modd_budget.f90*) :

```fortran
integer, parameter :: NLVL_ROOT        = 0
integer, parameter :: NLVL_CATEGORY    = 1
integer, parameter :: NLVL_SUBCATEGORY = 2
integer, parameter :: NLVL_GROUP       = 3
integer, parameter :: NLVL_SHAPE       = 4
integer, parameter :: NLVL_TIMEAVG     = 5
integer, parameter :: NLVL_NORM        = 6
integer, parameter :: NLVL_MASK        = 7
```


#### Bilans (budgets)

*  Le choix des paramètres, des variables et des termes sources se fait en utilisant la nameliste *NAM_BUDGET*.

*  Ajout d'un terme source
  *  Ajouter ses caractéristiques (nom et conditions pour sa disponibilité) dans *ini_budget.f90*
  *  Ajouter les appels à *Budget_store_init* et à *Budget_store_end*
  *  Alternative : *Budget_store_add*
  
Les bilans pour les différentes variables pronostiques peuvent être référencés à partir de *parameters* (voir fichier *MNH/modd_budget.f90*) :

```fortran
integer, parameter :: NBUDGET_RHO = 0  ! Reference number for budget of RhoJ
integer, parameter :: NBUDGET_U   = 1  ! Reference number for budget of RhoJu  and/or LES budgets with u
integer, parameter :: NBUDGET_V   = 2  ! Reference number for budget of RhoJv  and/or LES budgets with u
integer, parameter :: NBUDGET_W   = 3  ! Reference number for budget of RhoJw  and/or LES budgets with u
integer, parameter :: NBUDGET_TH  = 4  ! Reference number for budget of RhoJTh and/or LES budgets with th
integer, parameter :: NBUDGET_TKE = 5  ! Reference number for budget of RhoJTke and/or LES budgets with Tke
integer, parameter :: NBUDGET_RV  = 6  ! Reference number for budget of RhoJrv and/or LES budgets with rv
integer, parameter :: NBUDGET_RC  = 7  ! Reference number for budget of RhoJrc and/or LES budgets with rc
integer, parameter :: NBUDGET_RR  = 8  ! Reference number for budget of RhoJrr and/or LES budgets with rr
integer, parameter :: NBUDGET_RI  = 9  ! Reference number for budget of RhoJri and/or LES budgets with ri
integer, parameter :: NBUDGET_RS  = 10 ! Reference number for budget of RhoJrs and/or LES budgets with rs
integer, parameter :: NBUDGET_RG  = 11 ! Reference number for budget of RhoJrg and/or LES budgets with rg
integer, parameter :: NBUDGET_RH  = 12 ! Reference number for budget of RhoJrh and/or LES budgets with rh
integer, parameter :: NBUDGET_SV1 = 13 ! Reference number for 1st budget of RhoJsv and/or LES budgets with sv
```

##### Exemple : terme *VISC* pour la variable pronostique *UU*

Dans *ini_budget.f90* :
```fortran
tzsource%cmnhname   = 'VISC'
tzsource%clongname  = 'viscosity'
tzsource%lavailable = lvisc .and. lvisc_uvw
call Budget_source_add( tbudgets(NBUDGET_U), tzsource )
```

Dans *viscosity.f90* :
```fortran
if (lbudget_u .and. ovisc_uvw) &
  call Budget_store_init( tbudgets(NBUDGET_U), 'VISC', prus )
...
if (lbudget_u .and. ovisc_uvw) &
  call Budget_store_end( tbudgets(NBUDGET_U), 'VISC', prus )
```

#### Bilans LES

*  Le choix des paramètres, des variables et des termes sources se fait en utilisant la nameliste *NAM_LES*.

Les différents budgets sont :

*  déclarés dans les fichiers *modd_les.f90* et *modd_lesn.f90*,
*  alloués et initialisés dans *ini_lesn.f90*,
*  calculés dans *lesn.f90* (voir aussi *switch_sbg_lesn.f90*),
*  écrits dans les fichiers *write_les.f90*, *write_les_budgetn.f90*, *write_les_rt_budgetn.f90* ou *write_les_sv_budgetn.f90* selon le type budget.


#### *Flyers* (avions et ballons)

*  Les caractéristiques des avions et ballons doivent être fournies à l'aide des namelistes *NAM_FLYERS*, *NAM_AIRCRAFTS* et *NAM_BALLOON* (depuis Méso-NH 5.6.0, auparavant en modifiant les sources des fichiers *ini_aircraft.f90* et *ini_balloon.f90*).

*  L'ajout d'un champ à écrire se fait dans *write_aircraft_balloon.f90*


#### Profileurs

*  Les caractéristiques des profileurs sont fournies par la nameliste *NAM_PROFILERn* (depuis Méso-NH 5.6.0, auparavant dans le fichier *ini_profilern.f90*).

*  L'ajout d'un champ à écrire se fait dans *write_profilern.f90* avec
  *  un appel à *Add_profile* pour les données sous forme de profil vertical,
  *  un appel à *Add_point* pour les données ponctuelles au niveau du profileur.

#### Stations

*  Les caractéristiques des stations sont fournies à l'aide de la nameliste *NAM_STATIONn* (depuis Méso-NH 5.5.0, auparavant dans le fichier *ini_stationn.f90*).

*  L'ajout d'un champ à écrire se fait dans *write_stationn.f90*.

#### Séries temporelles

*  Pour les séries temporelles (données moyennées dans des boîtes cartésiennes), il faut passer par les namelistes *NAM_SERIES* et *NAM_SERIESn*.

*  L'ajout de nouvelles séries est décrit dans le guide de l'utilisateur de Méso-NH.

<a id='messages'></a>
### Messages

*  L'infrastructure de messages permet d'afficher des messages dans les fichiers de diagnostiques *OUTPUT_LISTINGn* ou sur la sortie standard.

*  Le choix de ses paramètres est fait dans *NAM_CONFIO*.

*  Cette infrastructure permet aussi de forcer la sortie de Méso_NH. C'est d'ailleurs l'approche à privilégier pour terminer l'exécution avant la fin car
  *  il y a aussi affichage du message de diagnostique sur l'erreur standard (en plus d'*OUTPUT_LISTINGn* et/ou la sortie standard) et
  *  la fin d'exécution est faite proprement (libération des ressources MPI entre autres).

```fortran
subroutine Print_msg_multi( KVERB, HDOMAIN, HSUBR, HMSG, OLOCAL )

INTEGER,                        INTENT(IN) :: KVERB   !Verbosity level
CHARACTER(LEN=*),               INTENT(IN) :: HDOMAIN !Domain/category of message
CHARACTER(LEN=*),               INTENT(IN) :: HSUBR   !Subroutine/function name
CHARACTER(LEN=*), dimension(:), INTENT(IN) :: HMSG    !Message
LOGICAL, OPTIONAL,              INTENT(IN) :: OLOCAL  !true to force print on this process (if verbosity level is high enough)
```

Rappel des niveaux de verbosité :

| # | Mot clé       | Description     |
| :- | :-           | :-              |
| 0 | NVERB_NO      | pas de messages |
| 1 | NVERB_FATAL   | erreurs fatales |
| 2 | NVERB_ERROR   | erreurs         |
| 3 | NVERB_WARNING | avertissements  |
| 4 | NVERB_INFO    | informations    |
| 5 | NVERB_DEBUG   | débogage        |

Actuellement, il y a 3 domaines (catégories) de messages possibles (argument muet *HDOMAIN*) :

*  BUD pour les bilans (*budgets*),
*  IO pour les entrées-sorties,
*  GEN pour tous les autres.

Exemple d'utilisation :
```fortran
use mode_msg

call Print_msg( NVERB_ERROR, 'GEN', 'NOM_SUBROUTINE',  'message d''erreur sur une seule ligne' )

cmnhmsg(1) = 'Message decoupe'
cmnhmsg(2) = 'sur plusieurs lignes'
cmnhmsg(3) = '(maximum 10 pour MesoNH 5.6.1)'
call Print_msg( NVERB_INFO, 'GEN', 'NOM_SUBROUTINE' )
```

Merci pour votre attention !

Des questions ?

## Evolutions / changelog (pour les I/O)

### 5.2.0 (25/01/2016)

* Entrées/sorties complètes en netCDF-4. Choix du format de fichier (LFI ou netCDF) dans la nameliste *NAM_CONFIO*

### 5.2.1 (10/05/2016)

* Pas de changements (pour les I/O)

### 5.2.2 (09/12/2016)

* Pas de changements (pour les I/O)

### 5.3.0 (16/12/2016)

* *LFI2CDF* : ajout de nouvelles options pour gérer les fichiers découpés par niveaux, compresser les données, réduire la précision pour les flottants, extraire des champs 3D dans des fichiers distincts
* création d'une nouvelle commande : *cdf2cdf*


### 5.3.1 (23/10/2017)

* XTSTEP_OUTPUT = n dans NAM_FMOUT pour spécifier une écriture toutes les n secondes

### 5.4.0 (02/07/2018)

* Formats de fichiers : netCDF par défaut (au lieu de LFI)
* Distinction entre *backups* (reprises) et *outputs* (sorties avec choix des champs)
* Respect conventions CF (netCDF uniquement)
* Ajout de métadonnées pour les différents champs (netCDF)
* Namelist NAM_FMOUT remplacée par NAM_BACKUP
* Namelist NAM_OUTPUT créée
* En interne, utilisation interface Fortran 95 de netCDF
* En interne, plus de vérifications et détection précoce de problèmes (incohérences, unités...)

### 5.4.1 (24/07/2018)

* Pas de changements (pour les I/O)

### 5.4.2 (08/04/2019)

* Nouveaux attributs dans les fichiers netCDF (history, MNH_cleanly_closed)
* Fichiers découpés verticalement :
    * Ajout d’un en-tête (similaire au fichier principal)
    * Retiré le stockage des champs scalaires (toujours présents dans fichier principal)
* Amélioration de la compatibilité avec les anciens fichiers (Méso-NH < 5.4)

### 5.4.3 (02/04/2020)

* Fichiers découpés verticalement : ajout d’informations sur le découpage des variables (dans fichier principal)
* *cdf2cdf/lfi2cdf* : amélioration de la conversion de tous les fichiers et de la fusion des fichiers découpés

### 5.4.4 (18/12/2020)

* Correction de nombreux bugs dans les bilans (budgets)

### 5.5.0 (21/07/2021)

* Réorganisation fichiers diachroniques
    * Fichiers diachroniques avec une structure hiérarchisée (niveaux) pour les fichiers netCDF (peu de changements pour le format LFI)
    * Ajout de nouvelles dimensions dans les fichiers netCDF
    * Namelistes simplifiées pour les bilans
    * Réécriture complète des sources
    
### 5.5.1 (13/06/2022)

* Pas de changements (pour les I/O)

### 5.6.0 (08/03/2023)


*  I/O management for scalar variables (users’, microphysics, electricity, chemistry, Lagrangian tracers, passive pollutants, salt, dust...)
    * New implementation to simplify their management and improve consistency between reads and writes
    * Correction of numerous bugs, inconsistencies and oversights (missing reads and/or writes)
    * Change in the naming, more easily identifiable
    * Physical units (mainly for chemistry) have been homogenized
    * Metadata has changed for some variables (units corrections, more details...).
    * When possible, backward compatibility with older Méso-NH files has been ensured.

* Sensors (aircraft, balloons, profilers and stations)
    * Use of namelists and CSV files (see user’s guide on how to use them) instead of modifying source code
    * Complete internal recoding
        * parallel performance optimisations (was very bad with large number of sensors)
        * memory use reduction (sensors are now allocated only locally and only when needed)
        * various improvements (field metadata, detection of abnormalities, bug corrections, more written fields...)

* For developers: custom metadata field constructor
    * Use of a custom constructor (Fortran 2003 functionality) for field metadata
    * Default values automatically set (depending on the context), coherence checks done

* Scalar logicals from SURFEX: names are suffixed with _SFX. Necessary change due to name collisions with Méso-NH variables (especially in chemistry simulations). Backward compatibility (in reading) with pre-5.6.0 Méso-NH files is ensured.
* NetCDF Climate and Forecast (CF) Metadata Conventions: use version 1.10 (instead of 1.7). Hierarchical structure is now part of CF.
* Others small changes

### 5.6.1 (27/09/2023)

* Sensors (aircrafts, balloons, stations and profilers)
    * Finalization of internal developments (object oriented in Fortran 2008, deduplication of code, communication optimisations...)
    * Added missing fields (ie dry density and plenty of others)
    * Homogenization of the written fields between the different sensor types
    
* Bug correction in parallel I/O (Z-split files): memory leak removed (was crashing after some time because of use of all available memory, impacting Méso-NH version 5.5.x and 5.6.0)

### 5.6.2 (28/11/2023)

* Bug correction for cyclic domains: 1D horizontal coordinates were wrong at domain boundaries

### 5.7.0 (xx/xx/202x)

* Lossy compression for output files
* Several fields have been added to the sensor outputs.
    * For aircraft and balloons, vertical profiles of temperature, pressure and scalar variables are now available.
    * Temperature at position of the flyer has also been added.
    * For profilers, vertical profiles of temperature and, for stations, evolution of temperature at their position are also written.
* The altitude profiles for profilers are no longer written with a time dimension as they do not change during simulations.



---

Notebook ayant servi de support pour une formation sur l'utilisation des entrées-sorties dans Méso-NH donnée le 17 mars 2022 en visioconférence.

Le document actuel est une mise à jour de ce dernier afin d'y intégrer les évolutions ayant eu lieu depuis.

Auteur : Philippe Wautelet, CNRS, Laboratoire d'aérologie, Toulouse

Notebook distribué sous [Licence Creative Commons Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/deed.fr) ![Licence Creative Commons (CC BY-NC-SA 4.0)](https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png)
